#  Simple Web Server with CORS
*When your browser refuses to load local files*

Sometimes we want our browser to be able to load a local file

1. .json
2. .js
3. ... etc

But the resource is blocked. Running a local <keyword>web server</keyword> is a quick workaround. There's several ways to spin one up - here I'll focus on using Python.

The [Python Docs Example](https://docs.python.org/3/library/http.server.html) will get you 80% there. However, you may encounter <keyword>CORS</keyword> Preflight checks failing.

You can read the spec on [MDN](https://developer.mozilla.org/en-US/docs/Web/HTTP/CORS#simple_requests) which goes a great amount of detail on the protocol. But for local development,
when we just want it to run we can fix these errors by adding one header to our response.


In [ ]:
import os
import argparse
import socketserver
from functools import partial
from http.server import SimpleHTTPRequestHandler

class CORSRequestHandler(SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        SimpleHTTPRequestHandler.end_headers(self)

if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    # Note that if you're making a request from 127.0.0.1 to localhost - CORS applies!
    parser.add_argument('--bind', '-b', default='', metavar='ADDRESS',
                        help='Specify alternate bind address '
                             '[default: all interfaces]')
    parser.add_argument('--directory', '-d', default=os.getcwd(),
                        help='Specify alternative directory '
                             '[default:current directory]')
    parser.add_argument('port', action='store',
                        default=8000, type=int,
                        nargs='?',
                        help='Specify alternate port [default: 8000]')
    args = parser.parse_args()
    print(f"Serving root at {args.directory}")
    handler_class = partial(CORSRequestHandler, directory=args.directory)
    with socketserver.TCPServer((args.bind, args.port), CORSRequestHandler) as httpd:
        httpd.serve_forever()

I keep a .bat file handy to quickly launch this for specific projects

```bat
CALL C:\path\to_your_venv\activate.bat
ECHO "Starting CORS Server"
cd C:\path\to_module
python server.py -d "C:\path\to_server_root\directory"
PAUSE
```
